In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch

In [2]:
# Parameter
Step = 0.01
Bp = 0.8
By = 0.318
Kpp = 0.204
Kyy = 0.072
Kpy = 0.0068
Kyp = 0.0219
Jp = 0.0178
Jy = 0.0084
l = 0.186
m = 1.3872
J_Tp = Jp + m*l**2
J_Ty = Jy + m*l**2
A = np.array([[0,0,1,0],[0,0,0,1],[0,0,-Bp/J_Tp,0],[0,0,0,-By/J_Ty]])
B = np.array([[0,0],[0,0],[Kpp/J_Tp,Kpy/J_Tp],[Kyp/J_Ty,Kyy/J_Ty]])
C = np.array([[1],[0],[1],[0]])

In [3]:
R = 0.2*np.eye(2)
Q = 2*np.eye(4)
gamma = 5

In [4]:
import itertools

In [5]:
X_sample = np.arange(-1,1.01,0.4)
X = np.array(list(itertools.product(X_sample,repeat=4)))

In [6]:
X[:,0]*X[:,1]

array([1., 1., 1., ..., 1., 1., 1.])

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.fc = nn.Linear(10,1,bias = False)
    def forward(self,x):
        x1 = x[0]
        x2 = x[1]
        x3 = x[2]
        x4 = x[3]
        X = torch.Tensor([x1**2,x1*x2,x1*x3,x1*x4,x2**2,x2*x3,x2*x4,x3**2,x3*x4,x4**2])
        output = self.fc(X)
        return output
model = Model()

In [8]:
def gradient_matrix(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    A = np.array([[2*x1,0,0,0],
                  [x2,x1,0,0],
                  [x3,0,x1,0],
                  [x4,0,0,x1],
                  [0,2*x2,0,0],
                  [0,x3,x2,0],
                  [0,x4,0,x2],
                  [0,0,2*x3,0],
                  [0,0,x4,x3],
                  [0,0,0,2*x4]])
    return A
def utility_fun(x,W):
    # R = 0.2*np.eye(2)
    # gamma = 5
    delta_phic = gradient_matrix(x)
    u = -1/2*np.linalg.pinv(R)@B.T@delta_phic.T@W
    u = u.flatten()
    w = 1/2/gamma**2*C.T@delta_phic.T@W
    r = x.T@Q@x + u.T@R@u - gamma**2*w.T@w
    r = r[0][0]
    K = C@w
    dx = A@x + B@u + K.flatten()
    x_new = x + Step*dx
    return r,x_new

# W = model.fc.weight.detach().numpy().T
# r,x_new = utility_fun(np.array([1,1,1,1]),W)

In [9]:
W = model.fc.weight.detach().numpy().T

In [10]:
from tqdm import tqdm
from torch.utils.data import DataLoader,TensorDataset
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)
for episode in range(20):
    print(f"EPOSODE {episode + 1}:")
    V_data = []
    for x in tqdm(X):
        r_data = []
        for i in range(100):
            r,x_new = utility_fun(x,W)
            r_data.append(r)
            x = x_new.copy()
        V = sum(r_data)*Step + model(torch.Tensor(x_new)).detach().numpy()[0]
        V_data.append(V)
    dataset = TensorDataset(torch.Tensor(X),torch.Tensor(V_data))
    train_loader = DataLoader(dataset,shuffle=True,batch_size=1)
    model.train()
    EPOCH = 2
    for epoch in range(EPOCH):
        losses = 0
        for (data,target) in train_loader:
            optimizer.zero_grad()
            output = model(data[0])
            loss = criterion(output,target)
            loss.backward()
            optimizer.step()
            losses += loss.item()
        print(f"EPOCH {epoch+1}: Loss = {losses/len(train_loader)}")
    print(W.T)

EPOSODE 1:


100%|██████████| 1296/1296 [00:15<00:00, 82.48it/s]


EPOCH 1: Loss = 0.0762334721427818
EPOCH 2: Loss = 1.0456142124976648e-13
[[ 4.091818    1.0682917   0.25987628  0.7219257   2.1796744  -0.01944051
   0.4823068   0.1020893  -0.03856067  0.4704598 ]]
EPOSODE 2:


100%|██████████| 1296/1296 [00:16<00:00, 78.42it/s]


EPOCH 1: Loss = 0.015449216669475348
EPOCH 2: Loss = 1.8885613237871355e-13
[[ 4.585227   -0.54328114  0.5564372   0.02067035  3.3369992  -0.04995291
   0.7757003   0.09653544 -0.01277334  0.25004554]]
EPOSODE 3:


...

EPOCH 1: Loss = 1.203392666722558e-13
EPOCH 2: Loss = 1.3372139472540347e-13
[[ 3.9349053  -0.91302025  0.50358444 -0.187308    3.746716   -0.08570459
   1.0055543   0.094831   -0.02217323  0.23438829]]
EPOSODE 19:


100%|██████████| 1296/1296 [00:19<00:00, 66.93it/s]


EPOCH 1: Loss = 1.186633096310701e-13
EPOCH 2: Loss = 1.3184241078851088e-13
[[ 3.9349053  -0.91302013  0.5035843  -0.18730782  3.7467165  -0.08570432
   1.0055544   0.09483085 -0.02217338  0.23438808]]
EPOSODE 20:


100%|██████████| 1296/1296 [00:27<00:00, 46.81it/s]


EPOCH 1: Loss = 1.369943253487699e-13
EPOCH 2: Loss = 1.4538421906680527e-13
[[ 3.9349048  -0.91301996  0.50358427 -0.18730791  3.746716   -0.08570459
   1.0055544   0.09483097 -0.02217333  0.23438811]]


In [12]:
np.save('W2.npy',W)